### Setting Model for The Experiment

The model that is going to be used for training can be set in the experiment using the method `set_model_class`.

In [ ]:
exp.set_training_plan_class(training_plan_class=MyTrainingPlan)
#exp.set_training_plan_path(training_plan_path=model_file)

<div class="note">
    <p>If you set your model path first, setter will log a debug message which will inform you about the model is not defined yet. This is because the model class has not been set yet</p>
</div>

### Setting The Argument Model Path (Special Case)
The `model_path` is the path your model is saved as a python script. This argument should be used if your model class is defined in different directory as python script. However, the experiment also need to now your class name. You can set your class name as a `string` with `set_model_class`. Since it is a python script (module), class name will be used for importing operation at the back-end.

```python
exp.set_model_path(model_path='path/to/your/script.py')
exp.set_model_class(model_class='ModelClassAsString')
```

### Setting Model and Training Arguments
In the previous step, the model has been defined for your experiment. Now, you can define your model arguments and training arguments that will be used respectively for building your model class and training your model on the node side. The methods `set_model_args` and `set_training_args` of the experiment class will allow you to set these arguments.

<div class="">
    <p>There isn't any requirement on the order of defining model class and mode/training arguments. It is also possible to
        define model/training arguments first and model class after. 
    </p>    
<div>


In [ ]:
# Model arguments should be an empty Dict, since our model does not require 
# any argument for initialization
model_args = {}

# Training Arguments
training_args = {
    'batch_size': 48, 
    'optimizer_args': {
        'lr': 1e-3, 
    },
    'num_updates': 100,
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

exp.set_model_args(model_args=model_args)
exp.set_training_args(training_args=training_args)

### Setting Tags

The tags for the dataset search request can be set using `set_tags` method of experiment object. 

<br><div class="note"><p>Setting tags does not mean sending dataset search request. Search request is sent while setting training data. `tags` is the argument that is required for the search request.</p></div>

The arguments `tags` of `set_tags` method should be an array of tags which are in `string` type or just a tag in `string` type.

In [ ]:
tags = ['#MNIST', '#dataset']
exp.set_tags(tags = tags)

To see the tags that are set, you can run `tags()` method of experiment object. 

In [ ]:
exp.tags()

### Setting Nodes
The `nodes` arguments indicates the nodes that are going to be used for the experiment. By default, it is equal to `None` which means every node up and running will be part of the experiment as long as they have the dataset that is going to be used for training. If the `nodes` has been set in advance, the search request for the dataset search will be sent only the nodes that are indicated. You can set nodes using the method `set_nodes(noes=nodes)`. This method takes `nodes` argument which should be an array of node ids which are in `string` type or just a single node id as `string`.

Since the node ids can change randomly, to make this notebook runnable in all environments, we won't be setting nodes for the experiment.


### Setting Training Data
Training data is a `FederatedDataset` instance which comes from the module `fedbiomed.researcher.datasets`. There are several ways to define your training data.

1. You can run `set_training_data(training_data=None, from_tags=True)`. This will send search request to the nodes to get dataset information by using the `tags` which are defined before.
2. You can provide `training_data` argument which is an instance of `FederatedDataSet`. 
3. You can provide `training_data` argument as python `dict` and setter will create a `FederatedDataSet` object by itself.

<div class="note"><p>While using the last option please make sure that your `dict` object is configured as coherent to `FederatedDataSet` schema. Otherwise, you might get error while running your experiment. </p></div>

If you run `set_training_data(training_data=None)`. No training data is defined yet for the experiment (`training_data` is set to `None`).


In [ ]:
training_data = exp.set_training_data(training_data=None, from_tags=True)

Since it will send search request to the nodes, the output will inform you about selected nodes for training. It means that those nodes have the dataset and able to train your model.

`set_training_data` will return a `FederatedDataSet` object. You can either use the return value of the setter or the getter for training data which is `training_data()`.

In [ ]:
training_data = exp.training_data()

To inspect the result in detail you can call the method `data()` of the `FederatedDataSet` object. This will return a python dictionary that includes information about the datasets that has been found in the nodes. 

In [ ]:
training_data.data()

As it is mentioned before, setting training data once doesn't mean that you can't change it. You can create a new `FederatedDataSet` with a `dict` that includes the information about the datasets. This will allow you to select the datasets that will be used for federated training.

<div class="note"><p>Since the dataset information will be provided, there will be no need to send request to the nodes</p></div>

In [ ]:
from fedbiomed.researcher.datasets import FederatedDataSet 

tr_data = training_data.data()
federated_dataset = FederatedDataSet(tr_data)
exp.set_training_data(training_data = federated_dataset)

Or, you can directly use `tr_data` in `set_training_data()`

In [ ]:
exp.set_training_data(training_data = tr_data)

<div class="note">
    <p>
        If you change the tags for the dataset by using <code>set_tags</code> and if there is already a defined training data in your experiment object, you have to update your training data by running <code>exp.set_training_data(training_data=None)</code>.  
    </p>
</div>

### Setting an Aggregator  

An aggregator is one of the required arguments for the experiment. It is used for aggregating model parameters that are received from the nodes after every round. By default, when the experiment is initialized without passing any aggregator, it will automatically use the default `FedAverage` aggregator class. However, it is also possible to set a different aggregation algorithm with the method `set_aggregator`. Currently, Fed-BioMed has only `FedAverage` but it is possible to create a custom aggregator classes.

You can see the current aggregator by running `exp.aggregator()`. It will return the aggregator object that will be used for aggregation. 

In [ ]:
exp.aggregator()

If we supposed that you have created your own aggregator, you can set it as follows,

In [ ]:
from fedbiomed.researcher.aggregators.fedavg import FedAverage
exp.set_aggregator(aggregator=FedAverage)

If your aggregator class needs initialization parameters, you can build your class and pass as an object .

In [ ]:
fed_average = FedAverage()
exp.set_aggregator(aggregator=fed_average)

### Setting Node Selection Strategy

Node selection Strategy is also one of the required arguments for the experiment. It is used for selecting nodes before each round of training. Since the strategy will be used for selecting nodes, before setting the strategy, training data should be already set. Then, strategy will be able to which nodes are current with their dataset.

By default, `set_strategy(node_selection_strategy=None)` will use the default `DefaultStrategy` class. It is default strategy that selects all the nodes available with their datasets at the moment. However, it is also possible to set different strategies. Currently, Fed-BioMed has only `DefaultStrategy` but you can create your custom strategy classes.



In [ ]:
exp.set_strategy(node_selection_strategy=None)

Or, you can directly pass `DefaultStrategy`

In [ ]:
from fedbiomed.researcher.strategies.default_strategy import DefaultStrategy
exp.set_strategy(node_selection_strategy=DefaultStrategy)

# To make the strategy has been set
exp.strategy()

### Setting Round Limit

Round limit is the limit that indicates max number of rounds of the training. By default, it is `None` and it needs to be set before running your experiment. You can set the round limit with the method `set_round_limit`. Round limit can  be changed after running one or several rounds of training. You can always execute `exp.round_limit()` to see current round limit.

In [ ]:
exp.set_round_limit(round_limit=2)
exp.round_limit()

### Setting Job to Manage Federated Training Rounds

Job is a class that manages federated training rounds. Before setting job, strategy for selecting nodes, model and training data should be set. Therefore, please make sure that they all defined before setting job.  The method `set_job` creates the Job instance and it does not take any argument. 

In [ ]:
exp.set_job()
exp.job()

### Setting Secure Aggregation

Secure aggregation enables nodes to send encrypted updates to the researcher. Thus researcher cannot read cleartext updates from the nodes. Nevertheless, researcher can aggregate updates from the nodes. The aggregated model parameters are then in cleartext, and the researcher can read them.

The method `use_secagg` toggles usage of secure aggregation. When `True`, a secure aggregation context (cryptographic material) for the experiment is negotiated between the experiment parties (if not existing yet) or existing secure aggregation context is re-used. Then usage of secure aggregation by the experiment is activated.

When negotiating a secagg context, `secagg_timeout` is the maximum setup time for the negotiation of each secure aggregation context element (server key and biprime). Thus maximum negotiation time is twice `secagg_timeout`.

In [ ]:
exp.set_use_secagg(use_secagg=True, timeout=10)

In [ ]:
print("Using secagg: ", exp.use_secagg())
exp_servkey, exp_biprime = exp.secagg_context()
if exp_servkey:
    print(f"Secagg servkey:\n- status {exp_servkey.status()}\n- secagg_id {exp_servkey.secagg_id ()}" \
        f"\n- context {exp_servkey.context()}")
else:
    print("No secagg servkey")
if exp_biprime:
    print(f"Secagg biprime:\n- status {exp_biprime.status()}\n- secagg_id {exp_biprime.secagg_id ()}" \
        f"\n- context {exp_biprime.context()}")
else:
    print("No secagg biprime")

In [ ]:
exp.set_use_secagg(False)

### Controlling Experiment Status Before Starting Training Rounds
Now, let's see if our experiment is ready for the training.

In [ ]:
exp.info()

If the experiment is ready, you will see the message that says `Experiment can be run now (fully defined)` at the bottom of the output. So now, we can run the experiment

## Running The Experiment

As long as `info()` says that the experiment is fully defined you will be able to run your experiment. Experiment has two methods  as `run()` and `run_once()` for running training rounds.

 - `run()` runs the experiment rounds from current round to round limit. If the round limit is reached it will indicate that the round limit has been reach. However, the method `run` takes to arguments as `round` and `increase`. 
    - `round` is an integer that indicates number of rounds that are going to be run. If the experiment is at round `0`, the round limit is `4`, and if you pass `round` as 3, it will run the experiment only for `3` rounds.
    - `increase` is a boolean that indicates whether round limit should be increased if the given `round` pass over the round limit. For example, if the current round is `3`, the round limit is `4`, and the `round` argument is `2`, the experiment will increase round limit to `5`
    
 - `run_once()` runs the experiment for single round of training. If the round limit is reached it will indicate that the round limit has been reach. However, if it is executed as `run_once(increase=True)` when the round limit is reach, it increases the round limit for one round.

In [ ]:
exp.run_once()

After running the experiment for once, you can check the current round. It returns `1` which means only one round has been run.

In [ ]:
exp.round_current()

Now, let's run the experiment with `run_once()` again. 

In [ ]:
exp.run_once()

Since the round limit has been set to `2` the round limit had been reached. If you try to run `run()` or `run_once()` the experiment will indicate that the round limit has been reached.

In [ ]:
exp.run_once()

In [ ]:
exp.run()

After this point, if you would like to run the experiment you can increase round limit with `set_round_limit(round)`

In [ ]:
exp.set_round_limit(4)
print('Round Limit    : ' , exp.round_limit())
print('Current Round  : ' , exp.round_current())

The round limit of the experiment has been set to `4` and the completed number of rounds is `2`. It means if you run the experiment with method `run()` without passing any argument, it will run the experiment for `2` rounds.

In [ ]:
exp.run()

Let's check the current round status of the experiment. 

In [ ]:
print('Round Limit    : ' , exp.round_limit())
print('Current Round  : ' , exp.round_current())

Another way to run our experiment if the round limit is reached is passing `rounds` to the method `run()`. For example, following cell will run the experiment for `2` more rounds.

In [ ]:
exp.run(rounds=2, increase=True) # increase is True by default

If the argument `increase` is `False`, it will not increase the round limit automatically. 

In [ ]:
exp.run(rounds=2, increase=False)

In [ ]:
print('Round Limit    : ' , exp.round_limit())
print('Current Round  : ' , exp.round_current())

It is also possible to increase number of rounds while running the experiment with `run_once()` by passing `increase` argument as `True`

In [ ]:
exp.run_once(increase=True)

In [ ]:
print('Round Limit    : ' , exp.round_limit())
print('Current Round  : ' , exp.round_current())

### Changing Training Arguments for the Next Round

The method `set_training_args()` allows you to change the training arguments even you've already run your experiment several times. Thanks to the method `set_training_args()` you will be able to configure your training from one round to another. For example, we can change our `batch_size` to `64` and `batch_maxnum` to `50` for the next round.


In [ ]:
# Training Arguments
training_args = {
    'batch_size': 64, 
    'optimizer_args': {
        'lr': 1e-3
    },
    'num_updates': 50,
    'dry_run': False,  
    'batch_maxnum': 50
}

exp.set_training_args(training_args=training_args)

In [ ]:
exp.run_once(increase=True)

### Conclusions 
The experiment class is the interface and the orchestrator of the whole processes behind federated training on the researcher side. It allows you to manage your federated training experiment easily. It has been extended with setter and getter methods to ease its declaration. This also provides more control before, during or after the training rounds. The purpose of the experiment class is to provide a robust interface for end-user to make them able to easily perform their federated training on Fed-BioMed nodes.